# Проект - Защита персональных данных клиентов

## Описание проекта
Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

print(df.head())
df.info()
df.describe()

   Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
0    1     41.0   49600.0            1                  0
1    0     46.0   38000.0            1                  1
2    0     29.0   21000.0            0                  0
3    0     21.0   41700.0            2                  0
4    1     28.0   26100.0            0                  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [3]:
# Корректируем тип данных в столбцах, это может ускорить процесс их обработки
df[['Возраст', 'Зарплата']] = df[['Возраст', 'Зарплата']].astype('int')



> Данные не имеют аномалий, пропущенных значений. Приведены к одному типу для ускорения работы.

# Ответьте на вопрос и обоснуйте решение. 
     Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
     Ответ: Не изменится. 

In [4]:
# Построим модель линейной регрессии
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

class LinearReg:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv((X.T @ X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]
        print(w)
        
    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

model = LinearReg()
model.fit(features, target)
predictions = model.predict(features)
r2_score(target, predictions)



[-9.38235447e-01  7.92580563e-03  3.57083050e-02 -1.70081903e-07
 -1.35676627e-02]


0.42494550308169177

## Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

In [12]:
# Создадим случайную кваратную матрицу, размером количества столбцов в features
random_matrix = np.random.normal(0, 1, (features.shape[1], features.shape[1]))

# Проверим сгенерированную матрицу на обратимость 
display('Единичная матрица:', abs(random_matrix @ np.linalg.inv(random_matrix)))

# Умножим признаки на обратимую матрицу 
encoded = features.dot(random_matrix)

#Заново обучаем и смотрим R2
model = LinearReg()
model.fit(encoded, target)
predictions = model.predict(encoded)
r2_score(target, predictions)



'Единичная матрица:'

array([[1.00000000e+00, 1.92464671e-15, 7.29716029e-18, 1.08075546e-16],
       [4.93730636e-17, 1.00000000e+00, 2.92732425e-17, 7.72592079e-19],
       [5.86169492e-17, 3.08108515e-16, 1.00000000e+00, 2.01242669e-17],
       [5.67703304e-16, 1.34518238e-15, 4.40309675e-18, 1.00000000e+00]])

[-0.9382372   0.15840625  0.42128275  0.17202907 -0.2532531 ]


0.42494550307804435

> Качество не изменилось

$w' = (X^T M^T XM)^-1  X^T M^T y$ \\ 
Предсказания: $ a = Wx$

Преобразованная матрица признаков получается умножением исходной $Х$ на случайную обратимую матрицу($М$), формула принимает вид $a' = XMw'$

Формула обучения в этом случае выглядит как $w' = ((XM)^T XM)^-1 (XM)^T y$

Раскрывая скобки принимает вид: 

$w' = (M^T X^T XM)^-1 M^T X^T y = M^-1(M^T X^T X)^-1 M^T X^T y = M^-1(X^T X)^-1 (M^T)^-1 M^T X^Ty$

В получившемся выражени $(M^T)^-1 M^T$ - единичная матрица, можно сократить:

И получим $w' = M^-1(X^T X)^-1 X^T y$

Исходная формула обучения $w = (X^T X)^-1 X^T y$

То есть новые веса можно выразить через исходные
$w' = M^-1 w$

Доказательство равенства предсказаний:

Формула предсказания :
$a = Xw$

Получая новую матрицу, мы умножаем ее на случайную обратную:
$a' = XMw'$

Ранее мы вычислили значение $w'$, так что подставим в формулу:
$a' = XM M^-1 w$

$M * M^-1$ -  единичная матрица, значит сокращаем:

$a' = Xw = a$

# Предложите алгоритм преобразования данных для решения задачи. 

In [6]:
def encode(features):
    # Создадим случайную кваратную матрицу, размером количества столбцов в features
    random_matrix = np.random.normal(0, 1, (features.shape[1], features.shape[1]))
    # Проверим сгенерированную матрицу на обратимость 
    display('Единичная матрица:', random_matrix @ np.linalg.inv(random_matrix))
    # Умножим признаки на обратимую матрицу 
    encoded = features.dot(random_matrix)
    return encoded

# Запрограммируйте этот алгоритм, применив матричные операции. 

Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2. 

In [7]:
# Обучаем модель из sklearn на исходных данных
sk_model = LinearRegression().fit(features, target)
sk_predictions = sk_model.predict(features)
r2_score(target, sk_predictions)


0.42494550308169177

In [8]:
# Обучаем модель из sklearn на кодированных данных
sk_model.fit(encoded, target)
pred = sk_model.predict(encoded)
r2_score(target, pred)


0.42494550308175716

> Качество моделей для закодированных и иходных данных аналогично. Качество также аналогично рукописной модели линейной регресии

# Общий вывод

* Защита данных не ухудшила показатель качества модели.
* Рукописная модель не уступает в качестве таковой из пакета sklearn